# Imports and Initialisation

In [202]:
import os
import configparser
import psycopg2
import folium
from folium.plugins import MarkerCluster

config = configparser.ConfigParser()
KEYS_DIR = os.path.join("D:\\Data", "API_Keys")
config.read(os.path.join(KEYS_DIR, "TPAMWeb.ini"))
db_pswd = config['MySQL']['p']

# Execute sql

In [203]:
def execute_sql(sql):
    try:
        with psycopg2.connect(
            host="localhost",
            port="5432",
            user="postgres",
            password=db_pswd,
            database="TravelGuide"
            ) as connection:

            with connection.cursor() as cursor:        
                cursor.execute(sql)
                result = cursor.fetchall()

    except Exception as e:
        print(e)

    finally:
        connection.close()

    return result

In [204]:
def map_points(results, marker_cluster, basemap, markers_only=False):
    for point in results:
        label_string = f'<a href="{point[1]} "target="_blank">{point[0]}</a>'
        label_html = folium.Html(label_string, script=True)
        label = folium.Popup(label_html, max_width=2650)
        folium.Marker(location = (point[2],point[3]), popup= label, tooltip=str(point[0])).add_to(marker_cluster)
        if not markers_only:
            folium.GeoJson(data=point[4]).add_to(basemap)

# SELECT the County

In [205]:
county = 'Bath and North East Somerset'
county = 'Bristol, City of'
sql = f"""SELECT ST_AsText(ST_CENTROID(geometry)) FROM public."UK_admin_boundaries" WHERE ctyua19nm = '{county}'"""

# Get the SRID of the table

In [206]:
sql = """SELECT Find_SRID('public', 'UK_admin_boundaries', 'geometry');"""
print(execute_sql(sql))
sql = """SELECT Find_SRID('public', 'gd_UK_parks_gardens', 'geometry');"""
print(execute_sql(sql))

[(4326,)]
[(4326,)]


# Get the Centroid of the County

In [207]:
sql = f"""SELECT ST_Y(ST_CENTROID(geometry)), ST_X(ST_CENTROID(geometry)) FROM public."UK_admin_boundaries" WHERE ctyua19nm = '{county}'"""
centroid = execute_sql(sql)
print(centroid)

[(51.45264560701666, -2.7706306012850526)]


# Create the Basemap Using the Centroid

In [208]:
  
import folium
# figure = folium.Figure()
basemap = folium.Map(centroid[0], zoom_start= 10, tiles='cartodb positron', prefer_canvas=True)
from folium.plugins import MarkerCluster
marker_cluster = MarkerCluster().add_to(basemap)  

# Get UK Parks & Gardens Full Records, Centroids as X,Y co-ordinate, Outline Geometries as GeoJSON
Note that selecting individual fiels (e.g. a.name) does not appear to work

In [209]:
sql = f""" 
SELECT a."Name", a."Hyperlink", ST_Y(ST_CENTROID(a.geometry)), ST_X(ST_CENTROID(a.geometry)), ST_AsGeoJSON(a.geometry) 
FROM 
public."gd_UK_parks_gardens" as a JOIN public."UK_admin_boundaries" as b 
ON ST_WITHIN(a.geometry, b.geometry) 
WHERE b.ctyua19nm = '{county}';
"""
results = execute_sql(sql)
# print(results[0])
map_points(results, marker_cluster, basemap)

# UK Listed Buildings

In [210]:
sql = f""" 
SELECT a."Name", a."Hyperlink", ST_Y(ST_CENTROID(a.geometry)), ST_X(ST_CENTROID(a.geometry)), ST_AsGeoJSON(a.geometry) 
FROM 
public."gd_UK_listed_buildings" as a JOIN public."UK_admin_boundaries" as b 
ON ST_WITHIN(a.geometry, b.geometry) 
WHERE b.ctyua19nm = '{county}'
AND
(a."Grade" = 'I' OR a."Grade" = 'II*');
"""
results = execute_sql(sql)
# print(results[0])
map_points(results, marker_cluster, basemap, markers_only=True)

# Schedule Monuments

In [211]:
sql = f""" 
SELECT a."Name", a."Hyperlink", ST_Y(ST_CENTROID(a.geometry)), ST_X(ST_CENTROID(a.geometry)), ST_AsGeoJSON(a.geometry) 
FROM 
public."gd_UK_scheduled_monuments" as a JOIN public."UK_admin_boundaries" as b 
ON ST_WITHIN(a.geometry, b.geometry) 
WHERE b.ctyua19nm = '{county}';
"""
results = execute_sql(sql)
print(results[0])
map_points(results, marker_cluster, basemap, markers_only=True)

("King's Weston Hill camp, Henbury", 'https://historicengland.org.uk/listing/the-list/list-entry/1002479', 51.500394900845585, -2.6410178289075086, '{"type":"Polygon","coordinates":[[[-2.640175985,51.500823552],[-2.640161757,51.500765248],[-2.640141986,51.500677383],[-2.640102005,51.500205356],[-2.640100937,51.50012897],[-2.640104251,51.500101168],[-2.640111411,51.500082615],[-2.64013357,51.500075543],[-2.640170456,51.500061459],[-2.640746254,51.499998652],[-2.640887301,51.49997961],[-2.640977648,51.499966554],[-2.641005723,51.499960124],[-2.641035853,51.499957451],[-2.641082105,51.499955939],[-2.641213038,51.499967784],[-2.641426407,51.499976659],[-2.641877116,51.499982975],[-2.641895364,51.499992928],[-2.641903681,51.500011711],[-2.641902275,51.500068671],[-2.641908014,51.500469762],[-2.641911983,51.500752694],[-2.641027845,51.500805263],[-2.640618007,51.500821091],[-2.640441041,51.50081946],[-2.640175985,51.500823552]]]}')


# Always Open Land

In [212]:
sql = f""" 
SELECT a."Name", '' AS Hyperlink, ST_Y(ST_CENTROID(a.geometry)), ST_X(ST_CENTROID(a.geometry)), ST_AsGeoJSON(a.geometry) 
FROM 
public."gd_UK_always_open_land" as a JOIN public."UK_admin_boundaries" as b 
ON ST_WITHIN(a.geometry, b.geometry) 
WHERE b.ctyua19nm = '{county}';
"""
results = execute_sql(sql)
print(results[0])
map_points(results, marker_cluster, basemap)

('Blaise Hamlet', '', 51.507230062152615, -2.6355556114847625, '{"type":"Polygon","coordinates":[[[-2.634973796,51.507289832],[-2.634964668,51.507274151],[-2.634856968,51.507312309],[-2.634783508,51.507215177],[-2.634855303,51.507192225],[-2.634939825,51.50722179],[-2.635096573,51.507250963],[-2.635143956,51.507198211],[-2.63523933,51.507145108],[-2.635359653,51.507152006],[-2.635528205,51.507158552],[-2.635539252,51.507098536],[-2.635644229,51.507001245],[-2.635809553,51.506955043],[-2.635954775,51.50694131],[-2.636028015,51.506979745],[-2.636133162,51.507082817],[-2.636124237,51.50719289],[-2.635980033,51.507271428],[-2.63577349,51.507343742],[-2.635617871,51.507344587],[-2.635494365,51.50741654],[-2.635505987,51.507500613],[-2.635298993,51.507540569],[-2.635120877,51.507418566],[-2.63505773,51.507354098],[-2.634973796,51.507289832]]]}')


In [213]:
basemap